# Hands-on Session with Problog
## Artificial Intelligence Fundamentals 2023/24
Elia Piccoli (elia.piccoli@phd.unipi.it)

---

We can reuse the virtual environment created in the previous session.

To recall, the packages needed are: _nle_, _minihack_, _matplotlib_, _notebook_, _pyswip_

For today lesson we will use _problog_ - https://problog.readthedocs.io/en/latest

### >> **Install** <<
Please read carefully the install instructions!

You can find them here: https://problog.readthedocs.io/en/latest/install.html

---

### _Introduction_

##### _Probabilistic facts_

The main difference between Prolog and ProbLog is that ProbLog support probabilistic facts. In the language, this extension is realized by the addition of a single operator `::`

This indicates that the fact heads is true with probability 0.5 and false with probability 1-0.5.

`0.5::heads.`

##### _Annotated disjunctions_

ProbLog also supports non-binary choices. For example, we can model the throw of die as follows.

This type of statement is called an _annotated disjunction_. It expresses that _at most_ one of these choices is true. 

`1/6::die(D, 1); 1/6::die(D, 2); 1/6::die(D, 3); 1/6::die(D, 4); 1/6::die(D, 5); 1/6::die(D, 6).`

##### _Probabilistic clauses_

ProbLog also supports probabilities in the head of clauses.

This means that if burglary is _true_, alarm will be true as well with 90% probability.

`0.1::burglary.`

`0.9::alarm :- burglary.`

##### _Queries_

A query indicates for which entity we want to compute the probability.

Queries are specified by adding a fact `query(Query)`


`0.5::heads(C).`

`two_heads :- heads(c1), heads(c2).`

`query(two_heads).`

##### _Evidence_

Evidence specifies any observations on which we want to condition this probability. Evidence conditions a part of the program to be true or false.

It can be specified using a fact `evidence(Literal)`.

`0.5::heads(C).`

`two_heads :- heads(c1), heads(c2).`

`evidence(\+ two_heads).`

`query(heads(c1)).`

---

### _Moving to Python_

In [ ]:
from problog.program import PrologString
from problog import get_evaluatable

We define our knowledge base as string.

In [ ]:
p = PrologString("""
coin(c1).
coin(c2).
0.4::heads(C); 0.6::tails(C) :- coin(C).
win :- heads(C).
query(win).
""")

We can compute the output of the queries by using the following command.

In [ ]:
get_evaluatable().create_from(p).evaluate()
# probability of win is that head comes up on coin c1 (first try) or c2 (second try): 0.4 + 0.6*0.4 = 0.64

A more in depth explanation of what is happening behind the scenes is given by the step-by-step solution.

In [ ]:
from problog.program import PrologString
from problog.formula import LogicFormula, LogicDAG
from problog.ddnnf_formula import DDNNF
from problog.cnf_formula import CNF

In [ ]:
# ground the program
lf = LogicFormula.create_from(p)

# break cycles in the ground program
dag = LogicDAG.create_from(lf)

# convert to CNF
cnf = CNF.create_from(dag)

# compile CNF to ddnnf
ddnnf = DDNNF.create_from(cnf)

ddnnf.evaluate()

Instead of providing the terms as string, it is possible to use python objects.

In [ ]:
from problog.program import SimpleProgram
from problog.logic import Constant, Var, Term, AnnotatedDisjunction

In [ ]:
coin,heads,tails,win,query = Term('coin'),Term('heads'),Term('tails'),Term('win'),Term('query')
C = Var('C')
p = SimpleProgram()
p += coin(Constant('c1'))
p += coin(Constant('c2'))
p += AnnotatedDisjunction([heads(C,p=0.4), tails(C,p=0.6)], coin(C))
p += (win << heads(C))
p += query(win)

get_evaluatable().create_from(p).evaluate()

---

### _Some more examples_

In [ ]:
t2 = PrologString("""
0.8::stress(ann).
0.4::stress(bob).
0.6::influences(ann,bob).
0.2::influences(bob,carl).

smokes(X) :- stress(X).
smokes(X) :- influences(Y,X), smokes(Y).

query(smokes(carl)).
query(smokes(bob)).
""")
get_evaluatable().create_from(t2).evaluate()
# probability that bob smokes is that Bob is tressed (0.4) or that he's not stressed (0.6) and Ann is stressed (0.8) and that Ann influences Bob (0.6)

In [ ]:
t3 = PrologString("""
0.4 :: heads.

0.3 :: col(1,red); 0.7 :: col(1,blue).
0.2 :: col(2,red); 0.3 :: col(2,green); 0.5 :: col(2,blue).

win :- heads, col(_,red).
win :- col(1,C), col(2,C).

query(heads).
query(win).
query(col(1,_)).
query(col(2,_)).
evidence(col(2,green)).
""")
get_evaluatable().create_from(t3).evaluate()
# win = 0.12 = 0.4*0.3 = probability of heads and red on color 1 assuming that color 2 is green

In [ ]:
t3 = PrologString("""
0.5::weather(sun,0) ; 0.5::weather(rain,0).
0.6::weather(sun,T) ; 0.4::weather(rain,T) :- T>0, Tprev is T-1, weather(sun,Tprev).
0.2::weather(sun,T) ; 0.8::weather(rain,T) :- T>0, Tprev is T-1, weather(rain,Tprev).
query(weather(_,10)).
""")
get_evaluatable().create_from(t3).evaluate()

---

#### _Today Hands-on_

_School Bayesian network_

![STUDENTS](student_img.png)

### _First task_

Define the bayesian network using ProbLog

In [ ]:
t = PrologString("""
0.6::difficult ; 0.7::smart.
0.85::success :- difficult, smart.
0.1::success :- difficult, \+smart.
0.98::success :- \+difficult, smart.
0.45::success :- \+difficult, \+smart.
query(success).
""")
get_evaluatable().create_from(t).evaluate()

![EXAMS](exams.png)

### _Second task_

Modify the Problog code in order to:

- Add the predicate _takes(S, C)_ that connects a student _student(S)_ to the course _course(C)_ taken.
- Modify the previous elements (_difficult(C), smart(S)_) to add the new clauses of the previous point (_student(S), course(C)_).
- Solve the following queries:
    - `query(difficult(_)).`
    - `query(smart(_)).`
    - `query(success(_,_)).`
- Specify the following evidences:
    - `evidence(difficult(c2)).`
    - `evidence(success(s1,c2)).`

### _Third task_

Try to remove _student(S), course(C)_ and connect _difficult(C), smart(S)_ only using the _takes(S, C)_ predicate.